In [23]:
import torch 
import torch.nn as nn
import torch.optim as optim 
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = load_breast_cancer()

X, y = data.data, data.target

# Split the training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

# Stndard Scaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Converting to PyTorch
X_train_tensors = torch.tensor(X_train, dtype=torch.float32)
y_train_tensors = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensors = torch.tensor(X_test, dtype=torch.float32)
y_test_tensors = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Define the Logisitc Regression Model
class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1) # Input feature is input dimensions and Output is still One
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

model = LogisticRegression(X_train.shape[1])
loss_fn = nn.BCELoss() # Bineary Cross Entropy ~ converting in range 0 - 1
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training the Model
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(X_train_tensors)
    loss = loss_fn(output, y_train_tensors)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epochs {epoch + 1}/{epochs}  | Loss: {loss.item(): .4f}")

with torch.no_grad():
    y_pred = model(X_test_tensors)
    y_pred_class = (y_pred > 0.5).float()
    accuracy = (y_pred_class.eq(y_test_tensors).sum()/y_test_tensors.shape[0]).item()
    print(f"Accuracy:{accuracy:.4f}")

Epochs 10/100  | Loss:  0.6260
Epochs 20/100  | Loss:  0.5083
Epochs 30/100  | Loss:  0.4348
Epochs 40/100  | Loss:  0.3850
Epochs 50/100  | Loss:  0.3492
Epochs 60/100  | Loss:  0.3222
Epochs 70/100  | Loss:  0.3009
Epochs 80/100  | Loss:  0.2837
Epochs 90/100  | Loss:  0.2694
Epochs 100/100  | Loss:  0.2573
Accuracy:0.9737


In [19]:
X_train, X_train.shape, X_train.shape[1]

(array([[9.029e+00, 1.733e+01, 5.879e+01, ..., 1.750e-01, 4.228e-01,
         1.175e-01],
        [2.109e+01, 2.657e+01, 1.427e+02, ..., 2.903e-01, 4.098e-01,
         1.284e-01],
        [9.173e+00, 1.386e+01, 5.920e+01, ..., 5.087e-02, 3.282e-01,
         8.490e-02],
        ...,
        [1.429e+01, 1.682e+01, 9.030e+01, ..., 3.333e-02, 2.458e-01,
         6.120e-02],
        [1.398e+01, 1.962e+01, 9.112e+01, ..., 1.827e-01, 3.179e-01,
         1.055e-01],
        [1.218e+01, 2.052e+01, 7.722e+01, ..., 7.431e-02, 2.694e-01,
         6.878e-02]], shape=(455, 30)),
 (455, 30),
 30)

In [48]:
# Make Prediciton;
for n in range(10):
    sm = X_test[n].reshape(1, -1)
    sm = scaler.transform(sm)
    sm = torch.tensor(sm, dtype=torch.float32)

    model.eval()
    with torch.no_grad():
        pred_prob = model(sm).item()
        pred_class = 1 if pred_prob > 0.5 else 0

    print(f'True Value: {y_test[n].item()}')
    print(50*"-")
    print(f'Prediciton Probablity', pred_prob)
    print(f'Predicted Class', pred_class)
    print(25*"- ")

True Value: 1
--------------------------------------------------
Prediciton Probablity 0.9999977350234985
Predicted Class 1
- - - - - - - - - - - - - - - - - - - - - - - - - 
True Value: 0
--------------------------------------------------
Prediciton Probablity 2.554641160124845e-23
Predicted Class 0
- - - - - - - - - - - - - - - - - - - - - - - - - 
True Value: 0
--------------------------------------------------
Prediciton Probablity 1.968224201513453e-15
Predicted Class 0
- - - - - - - - - - - - - - - - - - - - - - - - - 
True Value: 1
--------------------------------------------------
Prediciton Probablity 1.0
Predicted Class 1
- - - - - - - - - - - - - - - - - - - - - - - - - 
True Value: 1
--------------------------------------------------
Prediciton Probablity 1.0
Predicted Class 1
- - - - - - - - - - - - - - - - - - - - - - - - - 
True Value: 0
--------------------------------------------------
Prediciton Probablity 7.101386536104148e-13
Predicted Class 0
- - - - - - - - - - - 

In [ ]:
# Save the models;
from safetensors.torch import save_file

# Convert model state dict
model_state = model.state_dict()
# Save files 
save_file(model_state, 'Models/LR-BresastCancer.safetensors')
print('Model Save Sucessfully! (in safetensors)')

Model Save Sucessfully! (in safetensors)
